In [1]:
import pandas as pd
import numpy as np 
import matplotlib as plt
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

In [2]:
train_data = pd.read_csv("train.csv") 
test_data = pd.read_csv("test.csv")

train_data = train_data.drop(['PassengerId','Name','Ticket'], axis=1)
test_data = test_data.drop(['PassengerId','Name','Ticket'], axis=1)

full_data = [train_data, test_data]

submission_data = pd.read_csv("test.csv")

In [3]:
test_data["Fare"].fillna(test_data["Fare"].median(), inplace=True)

train_data["Age"].fillna(train_data["Age"].mean(), inplace=True)
test_data["Age"].fillna(test_data["Age"].mean(), inplace=True)

In [4]:
train_data.drop("Cabin",axis=1,inplace=True)
test_data.drop("Cabin",axis=1,inplace=True)

train_data['Embarked'] = train_data['Embarked'].fillna('S')
test_data['Embarked'] = test_data['Embarked'].fillna('S')

embark_dummies_titanic  = pd.get_dummies(train_data['Embarked'])
embark_dummies_titanic.drop(['S'], axis=1, inplace=True)
embark_dummies_test  = pd.get_dummies(test_data['Embarked'])
embark_dummies_test.drop(['S'], axis=1, inplace=True)

train_data = train_data.join(embark_dummies_titanic)
test_data  = test_data.join(embark_dummies_test)

train_data.drop(['Embarked'], axis=1,inplace=True)
test_data.drop(['Embarked'], axis=1,inplace=True)

In [5]:
def get_person(passenger):
    age,sex = passenger
    return 'child' if age < 16 else sex
    
train_data['Person'] = train_data[['Age','Sex']].apply(get_person,axis=1)
test_data['Person'] = test_data[['Age','Sex']].apply(get_person,axis=1)

train_data.drop(['Sex'],axis=1,inplace=True)
test_data.drop(['Sex'],axis=1,inplace=True)

person_dummies_titanic  = pd.get_dummies(train_data['Person'])
person_dummies_titanic.columns = ['Child','Female','Male']
person_dummies_titanic.drop(['Male'], axis=1, inplace=True)

person_dummies_test  = pd.get_dummies(test_data['Person'])
person_dummies_test.columns = ['Child','Female','Male']
person_dummies_test.drop(['Male'], axis=1, inplace=True)

train_data = train_data.drop(['Person'],axis = 1)
test_data = test_data.drop(['Person'],axis = 1)

train_data = train_data.join(person_dummies_titanic)
test_data    = test_data.join(person_dummies_test)

In [6]:
train_data['Family'] =  train_data["Parch"] + train_data["SibSp"]
train_data['Family'].loc[train_data['Family'] > 0] = 1
train_data['Family'].loc[train_data['Family'] == 0] = 0

test_data['Family'] =  test_data["Parch"] + test_data["SibSp"]
test_data['Family'].loc[test_data['Family'] > 0] = 1
test_data['Family'].loc[test_data['Family'] == 0] = 0

# drop Parch & SibSp
train_data = train_data.drop(['SibSp','Parch'], axis=1)
test_data    = test_data.drop(['SibSp','Parch'], axis=1)

/home/zad/anaconda3/envs/tf/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
X_train_data = train_data.drop("Survived",axis=1)
Y_train_data = train_data["Survived"]
X_test_data  = test_data

In [8]:
classifier = XGBClassifier()
classifier.fit(X_train_data, Y_train_data)

NameError: name 'XGBClassifier' is not defined

In [9]:
# Predicting the Test set results
y_pred = classifier.predict(X_test_data)



submission = pd.DataFrame({
        "PassengerId": submission_data["PassengerId"],
        "Survived": y_pred
    })
submission.to_csv('XGB1.csv', index=False)

NameError: name 'classifier' is not defined